In [ ]:
import speech_recognition as sr
import wave
import time
from datetime import datetime

import pyaudio
import pyttsx3

import openai

import os
import re

from dotenv import load_dotenv
load_dotenv()

In [ ]:
FORMAT        = pyaudio.paInt16
SAMPLE_RATE   = 44100        # サンプリングレート
CHANNELS      = 1            # モノラルかバイラルか
INPUT_DEVICE_INDEX = 0       # マイクのチャンネル
CALL_BACK_FREQUENCY = 3      # コールバック呼び出しの周期[sec]


OUTPUT_TXT_FILE = "./" + datetime.now().strftime('%Y%m%d_%H_%M') +".txt" # テキストファイルのファイル名を日付のtxtファイルにする

def callback(in_data, frame_count, time_info, status):
    """
    コールバック関数の定義
    """
    
    global sprec # speech_recognitionオブジェクトを毎回作成するのではなく、使いまわすために、グローバル変数で定義しておく

    try:
        audiodata  = sr.AudioData(in_data, SAMPLE_RATE, 2)
        sprec_text = sprec.recognize_google(audiodata, language='ja-JP')
        
        with open(OUTPUT_TXT_FILE,'a') as f: #ファイルの末尾に追記していく
            f.write("\n" + sprec_text)
    
    except sr.UnknownValueError:
        pass
    
    except sr.RequestError as e:
        pass
    
    finally:
        return (None, pyaudio.paContinue)


def realtime_textise():
    """
    リアルタイムで音声を文字起こしする
    """

    with open(OUTPUT_TXT_FILE, 'w') as f: #txtファイルの新規作成
        DATE = datetime.now().strftime('%Y%m%d_%H:%M:%S')
        f.write("日時 : " + DATE + "\n") # 最初の一行目に日時を記載する

    global sprec # speech_recognitionオブジェクトを毎回作成するのではなく、使いまわすために、グローバル変数で定義しておく
    
    # speech recogniserインスタンスを生成
    sprec = sr.Recognizer() 
    
    # Audio インスタンス取得
    audio  = pyaudio.PyAudio() 
    
    # ストリームオブジェクトを作成
    stream = audio.open(format             = FORMAT,
                        rate               = SAMPLE_RATE,
                        channels           = CHANNELS,
                        input_device_index = INPUT_DEVICE_INDEX,
                        input              = True, 
                        frames_per_buffer  = SAMPLE_RATE*CALL_BACK_FREQUENCY, # CALL_BACK_FREQUENCY 秒周期でコールバック
                        stream_callback    = callback)
    
    stream.start_stream()
    
    try:
        while stream.is_active():
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("end")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()


def main():
    #look_for_audio_input()
    realtime_textise()


if __name__ == '__main__':
    main()

In [24]:
# 発話クラスの定義
class SpeechModule:
    def __init__(self) -> None:
        self.engine = pyttsx3.init()
        self.voices = self.engine.getProperty('voices')
        self.engine.setProperty("voice", self.voices[1].id)

    def speech(self, text):
        self.engine.say(text)
        self.engine.runAndWait()

In [ ]:
speecher = SpeechModule()
speecher.speech("What should I do?")

In [59]:
# Chat-GPT
API_KEY = os.getenv('OPENAI_API_KEY')
class ChatGPT:
    def __init__(self) -> None:
        # ChatGPTの設定
        openai.api_key = API_KEY
        self.pattern =re.compile(r'[a-z]+:')
        
        # 発話機能の設定
        self.engine = pyttsx3.init()
        self.voices = self.engine.getProperty('voices')
        self.engine.setProperty("voice", self.voices[1].id)

    def request2gpt(self, request_text):
        start_sequence = "\nAI:"
        restart_sequence = "\nHuman: "

        response = openai.Completion.create(
            model="text-davinci-003",
            prompt="Human: "+request_text,
            temperature=0.9,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0.6,
            stop=[" Human:", " AI:"]
        )

        response_text = response['choices'][0]['text'].replace('\n', '').replace('Robot: ', '')
        response_text = self.pattern.sub('', response_text)
        return response_text
    
    def speech(self, text):
        self.engine.say(text)
        self.engine.runAndWait()
    
    def talk2gpt(self, request_text):
        res = self.request2gpt(request_text)
        print(res)
        self.speech(res)

gptkun = ChatGPT()

In [ ]:
gptkun.talk2gpt("Thanks")